In [108]:
import pandas as pd
import random, pickle

- display permet d'afficher des objets à la demande sans passer par print

In [109]:
from IPython.display import display

Conversion de nombre en pourcentage pour la lisibilité

In [110]:
from numbers import Number

def as_percent(v, precision='0.2'):  
    """Convert number to percentage string."""
    if isinstance(v, Number):
        return "{{:{}%}}".format(precision).format(v)
    else:
        raise TypeError("Numeric type required")

- définir les constantes
  - *million*, *debug*
  - facteurs de correction des fréquences :
    - *cFreqForme* => fréquence de la forme individuelle
    - *cFreqCases* => fréquence du lexème (Sg ou Pl)
    - *cFreqNull* => fréquence minimale

In [111]:
million=1000000
debug=False
cFreqForme=1000000
cFreqCases=1000
cFreqNull=1

- définir 
 - la taille des échantillons *nbMillions*
 - le nombre d'échantillons *nbSamples*
 - la référence du tirage

In [112]:
nbMillions=1
nbSamples=50
tiragePrefix="MGC-160804-1"
echantillon="-%dMo"%nbMillions

- choisir le type de fréquences à utiliser
 - fréquences brutes
 - fréquences ajustées
- choisir le jeu de cases à sélectionner
 - sans les formes rares : noSecondForms=True (pas de Ref to Nom, Nom2, Acc2, Inst2)
 - avec les formes rares : noSecondForms=True (avec toutes les cases de Uppsala)

In [113]:
freqsBrutes=False
noSecondForms=True

- lire le tableau de Uppsala dans un DataFrame
- renommer le Inst2Pl pour éliminer l'espace final en trop

In [114]:
pdRussianFreqs=pd.read_csv("/Users/gilles/ownCloud/Recherche/Boye/Russian-Data/Uppsala-Nouns.csv",sep=",",encoding="utf8")
pdRussianFreqs=pdRussianFreqs.rename(columns = {'Inst2Pl ':'Inst2Pl'})
pdRussianFreqs.groupby(by=["NM class","Animacy"])[["Frequency"]].count()

Frequency
NM class     Animacy           
DITJA        6                1
I            3               17
             4                4
             5              422
             6               15
             7               60
             8             1199
             9              565
II           3               24
             4                1
             5               71
             6                8
             7               36
             8              838
             9              633
III          3                3
             5                3
             7                5
             8              106
             9              353
IV           3                1
             5                1
             7                1
             8              235
             9              770
Indeclinable 5                1
             8               13
             9                1
ON_STEM      8                7
             9                2
PLURALIA     6                4
             8               35
             9                8
PLURALIA_I   8                1
PLURALIA_IV  8                3

- lister les colonnes
 - *lstTitres* : toutes les colonnes
 - *lstCases* : toutes les cases
 - *lstCasesMarginales*: les cases exceptionnelles
   - Ref to NOM
   - CAS2 sauf Loc2Sg
 - *lstCasesPrincipales*: les cases centrales

In [115]:
lstTitres=pdRussianFreqs.columns.tolist()
if debug: print lstTitres

lstCasesAll=lstTitres[6:-4]
if debug: print  len(lstCasesAll), lstCasesAll

if debug: print pdRussianFreqs[lstCasesAll].sum()

lstCasesMarginales=[case for case in lstCasesAll 
                    if "Ref" in case 
                    or case=="Vocative" 
                    or ("2" in case and case!="Loc2Sg")]
if debug: print lstCasesMarginales

lstCasesPrincipales=list(set(lstCasesAll)-set(lstCasesMarginales))
if debug: print lstCasesPrincipales

if noSecondForms:
    lstCases=lstCasesPrincipales
else:
    lstCases=lstCasesAll

if debug: print "cases considérées :",lstCases

### Calculer la fréquence ajustée
- séparer les singuliers des pluriels qui ont chacun une fréquence pour le lexème
- la fréquence ajustée est calculée avec trois termes :
  1. fréquence de la forme\*facteur forme
  1. fréquence du lexème (Sg ou Pl)\*fréquence case\*facteur lexème
  1. constante de fréquence minimale

In [116]:
def freqsAdjust(row):
#    case=row["case"].strip()
    if case.endswith("Sg") or case.endswith("Sg2") or case=="Vocative":
        lexFreq=row["Sg"]
    elif case.endswith("Pl") or case.endswith("Pl2"):
        lexFreq=row["Pl"]
    else:
        print "PB avec l'identité de la case :", [row["case"]]
    freqAdjusted=cFreqForme*row["rawFreq"]+cFreqCases*caseFreq[case]*lexFreq+cFreqNull
    return int(freqAdjusted)

### Calcul des fréquences de cases
- *caseFreqs* note le nombre d'occurrences de chaque case (pd.Series)
- *casesOcc* note le nombre d'occurrences totales (int)
- *caseFreq* note la fréquence de chaque case (dict)


In [117]:
caseFreqs=pd.DataFrame(pdRussianFreqs[lstCases].sum(),columns=["caseOcc"])
#caseFreqs

In [118]:
casesOcc=caseFreqs["caseOcc"].sum()
caseFreqs["caseFreq"]=caseFreqs["caseOcc"]/casesOcc
caseFreq=caseFreqs["caseFreq"].to_dict()
#caseFreq

- faire la liste des cases pour Sg et pour Pl pour gérer les fréquences de lexème séparément

In [119]:
lstCasesSg=[case for case in lstCases if not "Pl" in case]
lstCasesPl=[case for case in lstCases if "Pl" in case]
print lstCasesSg,lstCasesPl

[u'InstSg', u'AccSg', u'Loc2Sg', u'LocSg', u'GenSg', u'DatSg', u'NomSg'] [u'AccPl', u'LocPl', u'InstPl', u'NomPl', u'DatPl', u'GenPl']


- calculer les fréquences de cases séparément pour Sg et Pl

In [120]:
casesOccSg=caseFreqs[caseFreqs.index.isin(lstCasesSg)]["caseOcc"].sum()
casesOccPl=caseFreqs[caseFreqs.index.isin(lstCasesPl)]["caseOcc"].sum()
caseFreqs.loc[caseFreqs.index.isin(lstCasesSg),"caseFreq"]=caseFreqs["caseOcc"]/casesOccSg
caseFreqs.loc[caseFreqs.index.isin(lstCasesPl),"caseFreq"]=caseFreqs["caseOcc"]/casesOccPl
#caseFreqs[caseFreqs[lstCasesPl]]["caseFreq"]=caseFreqs["caseOcc"]/casesOccPl
caseFreq=caseFreqs["caseFreq"].to_dict()
caseFreq

{u'AccPl': 0.016452140630305274,
 u'AccSg': 0.069078709179117084,
 u'DatPl': 0.050297867086458621,
 u'DatSg': 0.05479592575607637,
 u'GenPl': 0.37091397351606631,
 u'GenSg': 0.28840119717120033,
 u'InstPl': 0.090255270549742264,
 u'InstSg': 0.094470324153411694,
 u'Loc2Sg': 0.0077478216794187923,
 u'LocPl': 0.087292033212951814,
 u'LocSg': 0.11800946151432502,
 u'NomPl': 0.3847887150044757,
 u'NomSg': 0.36749656054645075}

In [121]:
caseFreqs["case%"]=caseFreqs["caseFreq"].apply(as_percent)
print "Proportion des cas au Sg"
display(caseFreqs[caseFreqs.index.isin(lstCasesSg)].sort_values(by="caseFreq",ascending=False))
print
print "Proportion des cas au Pl"
display(caseFreqs[caseFreqs.index.isin(lstCasesPl)].sort_values(by="caseFreq",ascending=False))

Proportion des cas au Sg


,caseOcc,caseFreq,case%
NomSg,60903,0.367497,36.75%
GenSg,47795,0.288401,28.84%
LocSg,19557,0.118009,11.80%
InstSg,15656,0.094470,9.45%
AccSg,11448,0.069079,6.91%
DatSg,9081,0.054796,5.48%
Loc2Sg,1284,0.007748,0.77%



Proportion des cas au Pl


,caseOcc,caseFreq,case%
NomPl,24932,0.384789,38.48%
GenPl,24033,0.370914,37.09%
InstPl,5848,0.090255,9.03%
LocPl,5656,0.087292,8.73%
DatPl,3259,0.050298,5.03%
AccPl,1066,0.016452,1.65%


## Mettre le tableau en forme de lexique
- décroiser le tableau en liste de formes
- ajouter une colonne avec la fréquence ajustée
- calculer les fréquences cumulées et le max pour le tirage aléatoire

### Formater le tableau
  - décroiser le tableau
  - renommer les colonnes => case, rawFreq

In [122]:
lexique=pd.melt(pdRussianFreqs[["Gloss"]+lstCases+["Sg","Pl"]],id_vars=["Gloss","Sg","Pl"])
lexique=lexique.rename(columns = {'variable':'case',"value":"rawFreq"})

In [123]:
lexique=pd.melt(pdRussianFreqs[["Gloss","NM class","Animacy"]+lstCases+["Sg","Pl"]],id_vars=["Gloss","NM class","Animacy","Sg","Pl"])
lexique=lexique.rename(columns = {'variable':'case',"value":"rawFreq"})

### Ajouter les fréquences ajustées


In [124]:
lexique["adjFreq"]=lexique.apply(freqsAdjust,axis=1)
#lexique

### Ajouter les fréquences fréquences cumulées
- fréquences cumulées
- max


In [125]:
if freqsBrutes:
    lexique["freqcum"]=lexique.rawFreq.cumsum()
else:
    lexique["freqcum"]=lexique.adjFreq.cumsum()
nbOccurrences=lexique.freqcum.max()
nbOccurrences

231310848615

### Incrémentation des formes choisies par le tirage
- tri du tirage
- identification de la forme courante
  - incrémentation de la forme courante

In [126]:
def tirage2triage(tirage):    
    triage=sorted(tirage)
#    print triage[0:10]
    freqTop=0
    indexMin=0
    tirs={}

    for num,tir in enumerate(triage[:]):
        if tir > freqTop:
            indexMin=lexique[lexique["freqcum"]>=tir].iloc[0].name
            freqTop=lexique.ix[indexMin,'freqcum']
            tirs[indexMin]=0
        tirs[indexMin] += 1
        if debug and num%500000==0:
            print num,
    print
    for indexNum in tirs:
        lexique.ix[indexNum,'tir1']+=tirs[indexNum]

- on ajoute 1 à l'élément aléatoire pour ne pas démarrer avec un élément de fréquence nulle.

In [127]:
def creationSample(numero):
    lexique["tir1"]=0
    tirage=[]
    for n in range(nbMillions*million):
        tirage.append(random.randrange(nbOccurrences)+1)
    tirage2triage(tirage)
    with open("/Users/gilles/ownCloud/Recherche/Boye/Russian-Data/"+tiragePrefix+echantillon+"-%d"%numero+'-Tirage.pkl', 'wb') as output:
        pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open("/Users/gilles/ownCloud/Recherche/Boye/Russian-Data/"+tiragePrefix+echantillon+"-%d"%numero+'-Nums.pkl', 'wb') as output:
        pickle.dump(tirage, output, pickle.HIGHEST_PROTOCOL)

In [128]:
for sample in range(nbSamples):
    print "Numéro :",sample
    creationSample(sample)

Numéro : 0

Numéro : 1

Numéro : 2

Numéro : 3

Numéro : 4

Numéro : 5

Numéro : 6

Numéro : 7

Numéro : 8

Numéro : 9

Numéro : 10

Numéro : 11

Numéro : 12

Numéro : 13

Numéro : 14

Numéro : 15

Numéro : 16

Numéro : 17

Numéro : 18

Numéro : 19

Numéro : 20

Numéro : 21

Numéro : 22

Numéro : 23

Numéro : 24

Numéro : 25

Numéro : 26

Numéro : 27

Numéro : 28

Numéro : 29

Numéro : 30

Numéro : 31

Numéro : 32

Numéro : 33

Numéro : 34

Numéro : 35

Numéro : 36

Numéro : 37

Numéro : 38

Numéro : 39

Numéro : 40

Numéro : 41

Numéro : 42

Numéro : 43

Numéro : 44

Numéro : 45

Numéro : 46

Numéro : 47

Numéro : 48

Numéro : 49

